# Implementation of a SQP for nonlinear optimal control
The goal of this exercise is to implement a SQP solver to solve a nonlinear optimal control problem.

Consider the pendulum below 

<img src='pendulum.png' width="150">

Assuming $m=l=1$, The dynamics of this pendulum is
$$\ddot{\theta} = u - g \sin\theta$$
which can be discretized with the following dynamics
$$\begin{align}\theta_{n+1} &= \theta_n + \Delta t \omega_n\\ 
\omega_{n+1} &= \omega_n + \Delta t (u_n - g \sin\theta_n)\end{align}$$
where $\theta_n$ is the angle of the pendulum with respect to the vertical at time step $n$ and $\omega_n$ its angular velocity. We will use $\Delta t = 0.01$.
The pendulum starts at configuration $\theta_0 = \omega_0 = 0$, i.e. all the way down with zero velocity and we would like to find
an optimal control that will bring it up to $\theta=\pi$ with zero velocities.

To get the pendulum to do this movement, we write the following optimal control problem
$$\begin{align}
& \min_{\theta_n, \omega_n, u_n} \sum_{n=0}^{300} 10(\theta_n - \pi)^2 + 0.1\omega_n^2 + 0.1u_n^2\\
\textrm{subject to}\ \ & \theta_{n+1} = \theta_n + \Delta t \ \omega_n \\
& \omega_{n+1} = \omega_n + \Delta t\ (u_n - g \sin\theta_n)\\
& \theta_0 = \omega_0 = 0
\end{align}$$



## Question 1: write a SQP solver to solve this problem
To do so, please follow these steps:
* Write down the algorithm (in words not in code), i.e. write all the steps you need to take
* Write (in Latex) the gradient of the running cost at a given guess $\bar{x} = [\bar{\theta}_0, \bar{\omega}_0, \bar{u}_0, \bar{\theta}_1, \bar{\omega}_1, \bar{u}_1, \dots, \bar{\theta}_{300}, \bar{\omega}_{300}, \bar{u}_{300}]^T$, i.e. for given values $\bar{\theta}_n, \bar{\omega}_n, \bar{u}_n$ and implement a function that computes it
* Write (in Latex) the Hessian of the running cost at a given guess $\bar{x}$, i.e. for given values $\bar{\theta}_n, \bar{\omega}_n, \bar{u}_n$ and implement a function that computes it
* Assume that the Hessian of the constraints is 0 (i.e. ignore the second order derivatives of the constraints)
* Write (in Latex) a linear approximation of the constraints at a given guess $\bar{x}$ in a form $G(\bar{x}) \Delta x = g(\bar{x})$ (don't forget the constant terms in g) where $\Delta x$ represents a small variation around $\bar{x}$ and implement a function that computes both $G$ and $g$.
* Use these functions to construct the inner linear KKT system that you will solve using Numpy's solve function (this should resemble the KKT system you built in the first homework)
* Implement a function that computes the amount of constraint violation, i.e. the sum of the absolute values of all the constraints (i.e. assuming constraints of the form $c(x) = 0$ we want to compute $|c(x)|$).
* Implement a filter linear search to test if a step should be accepted. You will implement the (simplified) filter line search explained below.
* Terminate the algorithm when you either reached the maximum number of iterations (e.g. 100) or when the KKT optimality conditions $\nabla_x L$ and $\nabla_\lambda L$ are close to 0, e.g. $10^{-4}$.


Once you have a solution, make sure to check that it satisfies the constraints! You can also use the function ``pendulum.animate_robot`` to display the pendulum motion. Please answer the following questions:
1. How many iterations did it take?
2. Plot the solution (angle, velocity and control)
3. Plot the amont of constraint violation per iteration of the solver
4. Plot the cost per iteration of the solver
5. Plot $\alpha$ for each iteration of the solver

### (Simple) filter linear search
Once you have a potential step $p_x$ and associated candidate Lagrange multipliers $p_\lambda$ (from the ``solve`` of the KKT system), you need to find a step $\alpha$ to update your guess of the solution $x_{guess}$ and the Lagrange multipliers $\lambda_{guess}$. We will accept a step that either reduces the amount of constraint violation or reduces the cost.

Let's denote $f(x)$ the cost at $x$ and $|c(x)|$ the amount of constraint violation at $x$. Initialize the variable $f_{best} = \infty$ and $c_{best}=\infty$ at the beginning of the SQP. 

Then do the following during the line search.
1. Set $\rho$ to a number between 0 and 1 (e.g. 0.5) and set $\alpha = 1$
2. If $f(x_{guess} + \alpha p_x) < f_{best}$ then set $f_{best} \leftarrow f(x_{guess} + \alpha p_x)$ and accept the step

   Or 

   If $|c(x_{guess} + \alpha p_x)| < c_{best}$ then set $c_{best} \leftarrow |c(x_{guess} + \alpha p_x)|$ and accept the step
3. If the step was not accepted set $\alpha \leftarrow \rho \alpha$ and go back to Step 2.
4. If the step was accepted update the guess $x_{guess} \leftarrow x_{guess} + \alpha p_x$ and the Lagrange multipliers $\lambda_{guess} \leftarrow (1-\alpha)\lambda_{guess} + \alpha p_{lambda}$

## Solution 1 


In [1]:
# Import necessary libraries
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython
from scipy.linalg import block_diag

from qpsolvers import solve_qp, Problem, solve_problem

import pendulum 

# Set the max number of iterations
N = 300 

# Given value of Delta t
delta_t = 0.01

# Gravity 
gravity = 9.81

# Tolerance for the algorithm
tol = 1e-4


### Algorithm 

Given below are the steps to be followed to implement an SQP for the given model with only equality constraints 
#### 1. Reformulating the cost function
Given the cost function, $$\min_{\theta_n, \omega_n, u_n} \sum_{n=0}^{300} 10(\theta_n - \pi)^2 + 0.1\omega_n^2 + 0.1u_n^2$$ It appears similar to one for tracking a desired trajectory. So we shall reformulate it by expanding the term that depends on $\theta_n$ and ignore the constant terms. This leads us to the cost function, 
$\begin{equation} \notag \min_{\theta_n, \omega_n, u_n} \sum_{n=0}^{300} 10\theta_n^2 - 20\theta_n\pi + 0.1\omega_n^2 + 0.1u_n^2 \end{equation}$ OR
$ \begin{equation} \notag \min_{x} \sum_{n=0}^{300} \left( \frac{1}{2} x_n^T Q x_n -  x_{des}^T Q x_n \right) \end{equation} $ Where, 
$\begin{align}
    \notag &x_n = \begin{bmatrix} \theta_n \\  \omega_n \\ u_n\end{bmatrix} && Q = \begin{bmatrix} 20 & 0 & 0 \\ 0 & 0.2 & 0 \\ 0 & 0 & 0.2 \end{bmatrix} &&& x_{des} = \begin{bmatrix} \pi \\ 0 \\ 0 \end{bmatrix}
\end{align}$ 
Please note that the weight $R$ that is used to penalise the control input $u$ has been included inside of the $Q$ matrix, which is why it has not been explicity written in the above equation.

Rewriting this in matrix form and using the appropriate given terms, we get, 
$$\notag  \min_{\bar{x}} \frac{1}{2} \bar{x}^T G \bar{x} + g^T \bar{x} $$
$\begin{align}
    \notag &\text{Where, } \quad \bar{x} = \begin{bmatrix} \bar{x}_0 \\ \bar{x}_1 \\ \bar{x}_2 \\ \vdots \end{bmatrix} &&G = \begin{bmatrix} Q & 0 & 0 & \cdots \\ 0 & Q & 0 & \cdots \\ 0 & 0 & Q & \cdots \\ \vdots & \vdots & \vdots & \ddots\end{bmatrix} &&&g^T = \begin{bmatrix} -x_{des}^T Q & -x_{des}^T Q & -x_{des}^T Q & \cdots \end{bmatrix} &&&&\text{with each} \quad \bar{x}_n = \begin{bmatrix} \bar{\theta}_n \\ \bar{\omega}_n \\ \bar{u}_n \end{bmatrix} \forall n \in \left[0, 300\right] \quad \text{and} \quad x_{des} = \begin{bmatrix} \pi \\ 0 \\ 0 \end{bmatrix}
\end{align}$


#### 2. Reformulating the equality constraints
We have been given the constraints, 
$\begin{align}
\notag \theta_0 &= 0 \\
\notag \omega_0 &= 0 \\
\notag \theta_{n+1} &= \theta_n + \Delta t \ \omega_n \\
\notag \omega_{n+1} &= \omega_n + \Delta t\ (u_n - g \sin\theta_n)
\end{align}$

As we can see, one of the constraints is non-linear. So we must first find a linear approximation of the constraints at a given guess $\bar{x}$ in a form $G(\bar{x}) \Delta x = g(\bar{x})$ where $\Delta x$ represents a small variation around $\bar{x}$. To do that, we shall perform the Taylor Expansion of the non-linear constraints around $\bar{x}$, which will be given by, 
$\begin{align} 
    \notag \Delta \theta_{n+1} &= \Delta \theta_{n} + \Delta t \Delta \omega_n \\
    \notag \Delta \omega_{n+1} &= \Delta \omega_{n} + \Delta t (\Delta u_n - g  \Delta \theta_n \cos \theta_n)
\end{align}$

Combining these constraints in the matrix form, we get,
$\begin{equation}\notag
    \begin{bmatrix} 1 & \Delta t & 0 & -1 & 0 & 0 \\ - \Delta t g \cos \bar{\theta_n} & 1 & \Delta t & 0 & -1 & 0 \end{bmatrix} \begin{bmatrix} \Delta \theta_n \\ \Delta \omega_n \\ \Delta u_n \\ \Delta \theta_{n+1} \\ \Delta \omega_{n+1} \\ \Delta u_{n+1} \end{bmatrix} = \begin{bmatrix} 0 \\ 0\end{bmatrix}
\end{equation}$

Now, we shall include the original, non-linear constraints to obtain, 
$\begin{equation}\notag
    \begin{bmatrix} 1 & \Delta t & 0 & -1 & 0 & 0 \\ - \Delta t g \cos \bar{\theta_n} & 1 & \Delta t & 0 & -1 & 0 \end{bmatrix} \begin{bmatrix} \Delta \theta_n \\ \Delta \omega_n \\ \Delta u_n \\ \Delta \theta_{n+1} \\ \Delta \omega_{n+1} \\ \Delta u_{n+1} \end{bmatrix} = \begin{bmatrix}  \theta_n + \Delta t \ \omega_n - \theta_{n+1} \\   \omega_n + \Delta t\ (u_n - g \sin\theta_n) - \omega_{n+1} \end{bmatrix}
\end{equation}$

Now, we shall include the linear constraints as well and stack all of them together to obtain sparse matrices $G(\bar{x})$ and $g(\bar{x})$, 

$\begin{align} 
    \notag G (\bar{x}) \Delta x &= g(\bar{x}) \\ 
    \notag \\  
    \notag \Rightarrow \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\ 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\ 1 & \Delta t & 0 & -1 & 0 & 0 & 0 & 0 & 0 & \cdots \\ - \Delta t g \cos \bar{\theta_0} & 1 & \Delta t & 0 & -1 & 0 & 0 & 0 & 0 & \cdots \\ 0 & 0 & 0 & 1 & \Delta t & 0 & -1 & 0 & 0 & \cdots \\ 0 & 0 & 0 & - \Delta t g \cos \bar{\theta_1} & 1 & \Delta t & 0 & -1 & 0 & \cdots \\ \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots\end{bmatrix}  \begin{bmatrix} \Delta \theta_0 \\ \Delta \omega_0 \\ \Delta u_0 \\ \Delta \theta_1 \\ \Delta \omega_1 \\ \Delta u_1 \\ \Delta \theta_2 \\ \Delta \omega_2 \\ \Delta u_2 \\ \vdots  \end{bmatrix} &= \begin{bmatrix} 0 \\ 0 \\ \bar{\theta}_0 + \Delta t \ \bar{\omega}_0 - \bar{\theta}_1 \\   \bar{\omega}_0 + \Delta t\ (\bar{u}_0 - g \sin\bar{\theta}_0) - \bar{\omega}_1 \\ \bar{\theta}_1 + \Delta t \ \bar{\omega}_1 - \bar{\theta}_2 \\   \bar{\omega}_1 + \Delta t\ (\bar{u}_1 - g \sin\bar{\theta}_1) - \bar{\omega}_2 \\ \vdots \end{bmatrix}
\end{align}$

Based on this logic, the below function was written to compute the values of $G(\bar{x})$ and $g(\bar{x})$ with a given guess $\bar{x}$.

In [2]:
def compute_G_g(bar_x: np.ndarray):
    """
    Compute the Jacobian matrix G and the residual vector g for the linear approximation
    of the constraints in the Sequential Quadratic Programming (SQP) method.

    This function linearizes the nonlinear constraints of the pendulum optimal control problem
    around the current guess `bar_x`. The constraints include the initial conditions and
    the dynamic equations governing the pendulum's motion.

    Args:
        bar_x (np.ndarray): 
            Current guess of the state vector, structured as follows:
            [theta_0, omega_0, u_0, theta_1, omega_1, u_1, ..., theta_N, omega_N]
            - theta_n: Angle of the pendulum at time step n
            - omega_n: Angular velocity at time step n
            - u_n: Control input at time step n
            The length of `bar_x` should be `3 * N`, where N is the number of time steps.

    Returns:
        G (np.ndarray): 
            Jacobian matrix of the constraints with respect to the variables.
            Shape: (2 * N + 2, 3 * N)
            Each row corresponds to a constraint, and each column corresponds to a variable
            in `bar_x`. The matrix is structured to accommodate the initial conditions and
            the dynamic constraints for each time step.

        g (np.ndarray): 
            Residual vector of the constraints evaluated at `bar_x`.
            Shape: (2 * N + 2, 1)
            Each entry represents the negative of the constraint function evaluated at
            the current guess `bar_x`. This vector is used to form the linearized
            constraints in the SQP algorithm.
    """
    # Initialize the Jacobian matrix G as an identity matrix for the initial constraints
    # Shape: (2, 3 * N)
    G = np.eye(2, 3 * N)

    # Initialize the residual vector g with zeros for the initial constraints
    # Shape: (2, 1)
    g = np.zeros((2, 1), dtype=np.float64)

    # Set the first element of G corresponding to theta_0 = 0
    G[0, 0] = 1

    # Set the second element of G corresponding to omega_0 = 0
    G[1, 1] = 1

    # Loop over each time step to add dynamic constraints
    # The loop increments by 3 because variables are ordered as [theta, omega, u]
    for i in range(0, (N - 1) * 3, 3):
        # Extract the current state and control from bar_x
        bar_theta_i = bar_x[i]       # theta_n
        bar_omega_i = bar_x[i + 1]   # omega_n
        bar_u_i = bar_x[i + 2]       # u_n
        bar_theta_ip1 = bar_x[i + 3]  # theta_{n+1}
        bar_omega_ip1 = bar_x[i + 4]  # omega_{n+1}

        # Construct the Jacobian submatrix for the current dynamic constraints
        G_i = np.hstack((
            # Zeros for variables before the current time step
            np.zeros((2, (i // 3) * 3)),

            # Partial derivatives for the dynamic constraints at time step n
            np.array([
                [1, delta_t, 0, -1, 0, 0],  # Derivatives for theta constraint
                [
                    # Derivative w.r.t theta_n for omega constraint
                    -(delta_t * gravity * np.cos(bar_theta_i)),
                    1,
                    delta_t,
                    0,
                    -1,
                    0
                ]  # Derivatives for omega constraint
            ], dtype=np.float64),

            # Zeros for variables after the current time step
            np.zeros((2, (N - 2 - (i // 3)) * 3))
        ))

        # Vertically stack the new constraints into the Jacobian matrix G
        G = np.vstack((G, G_i))

        # Compute the residuals for the current dynamic constraints
        g_i = np.array((
            [
                bar_theta_i + (delta_t * bar_omega_i) - bar_theta_ip1
            ],  # Residual for theta constraint: theta_n + delta_t * omega_n - theta_{n+1}
            [
                bar_omega_i + \
                (delta_t * (bar_u_i - (gravity * np.sin(bar_theta_i)))) - bar_omega_ip1
            ]  # Residual for omega constraint: omega_n + delta_t * (u_n - g*sin(theta_n)) - omega_{n+1}
        ))

        # Vertically stack the new residuals into the residual vector g
        g = np.vstack((g, g_i))

    return G, g

#### 3. Forming the Lagrangian
Now that we have the basic requirments for constrained optimisation with equality constraints, we shall form the Lagrangian as shown below 
$$ \mathcal{L}(x, \lambda) = f(x) + \lambda ^ T g(x) $$

Where, 
$$f(x) = \frac{1}{2} \bar{x}^T G \bar{x} + g^T \bar{x} \quad \text{and} \quad g(\bar{x}) = G(\bar{x}) \Delta x$$
Please note that although the variables $G$ and $g$ are repeated, they have different meanings.

Using this, we can write the KKT Conditions to be, 
$\begin{align}
    \notag \nabla f(x) + \lambda \nabla g(x) &= 0 \\
    \notag g(x) &= 0
\end{align}$

And all terms have been defined in the previous steps. 

#### 4. Finding the gradient of the cost

Before we move on to solving the SQP, we need to do a couple of more steps, one of which is to find the gradient of the cost function, $$f(x) = \frac{1}{2} \bar{x}^T G \bar{x} + g^T \bar{x}$$ 

Let us recall that in lecture 2, a shortcut method was taught to find the gradient of a function. If a function is of the form, 
$\begin{align}
    \notag f(x) &= x^T P x + q^T x \\
    \notag \Rightarrow \nabla f(x) &= 2Px + q \\
    \notag \text{and} \quad \nabla ^2 f(x) &= 2 P
\end{align}$

By applying this trick to our equation, we can see that, 
$\begin{align}
\notag \nabla f(\bar{x}) &= 2 \cdot \frac{1}{2} G \bar{x} + g \\
\notag \Rightarrow \nabla f(\bar{x}) &= G \bar{x} + g \\
\end{align}$
$\begin{align}
    \notag &\text{Where, } \quad \bar{x} = \begin{bmatrix} \bar{x}_0 \\ \bar{x}_1 \\ \bar{x}_2 \\ \vdots \end{bmatrix} &&G = \begin{bmatrix} Q & 0 & 0 & \cdots \\ 0 & Q & 0 & \cdots \\ 0 & 0 & Q & \cdots \\ \vdots & \vdots & \vdots & \ddots\end{bmatrix} &&&g = \begin{bmatrix} (-x_{des}^T Q)^T \\ (-x_{des}^T Q)^T \\ (-x_{des}^T Q)^T \\ \vdots \end{bmatrix} &&&&\text{with each} \quad \bar{x}_n = \begin{bmatrix} \bar{\theta}_n \\ \bar{\omega}_n \\ \bar{u}_n \end{bmatrix} \forall n \in \left[0, 300\right] , \quad x_{des} = \begin{bmatrix} \pi \\ 0 \\ 0 \end{bmatrix} , \quad Q = \begin{bmatrix} 20 & 0 & 0 \\ 0 & 0.2 & 0 \\ 0 & 0 & 0.2 \end{bmatrix}
\end{align}$

The below function employs this logic to compute the gradient of the running cost.

In [3]:
def cost_grad(bar_x: np.ndarray):
    """
    Compute the gradient of the cost function for the optimal control problem.

    The cost function is defined as:
        f(x) = (1/2) * x^T * G * x + g^T * x
    where:
        - G is a block diagonal matrix with N copies of matrix Q along its diagonal.
        - g is a column vector where each 3-element block is (-x_des^T * Q)^T.

    This function calculates the gradient ∇f(x) = G * x + g.

    Args:
        bar_x (np.ndarray):
            Current guess of the state vector, structured as follows:
            [theta_0, omega_0, u_0, theta_1, omega_1, u_1, ..., theta_N, omega_N]
            - theta_n: Angle of the pendulum at time step n
            - omega_n: Angular velocity at time step n
            - u_n: Control input at time step n
            The length of `bar_x` should be `3 * N`.

    Returns:
        np.ndarray:
            The gradient of the cost function evaluated at `bar_x`.
            Shape: (3 * N, 1)
            Each element corresponds to the partial derivative of the cost
            function with respect to the corresponding state variable in `bar_x`.
    """
    # Define the weighting matrix Q
    Q = np.array([
        [20, 0, 0],
        [0, 0.2, 0],
        [0, 0, 0.2]
    ])

    # Define the desired state vector x_des
    x_des = np.array([
        [np.pi],
        [0],
        [0]
    ])

    # Construct the block diagonal matrix G with N copies of Q along the diagonal
    # Shape of G: (3*N, 3*N)
    G = block_diag(*([Q] * N))

    # Compute the vector g by repeating (Q @ x_des) N times
    # (Q @ x_des) results in a (3, 1) vector
    # After flattening and tiling, g has shape (3*N, 1)
    g = np.tile((Q @ x_des).flatten(), N).reshape(-1, 1)

    # Compute and return the gradient of the cost function: ∇f(x) = G * bar_x + g
    # bar_x is expected to be a (3*N, 1) vector
    return ((G @ bar_x) + g)

#### 5. Finding the hessian of the Lagrangian

This is the final step before solving the QP. We shall find the hessian of the Lagrangian formulated as, 
$$ \mathcal{L}(x, \lambda) = f(x) + \lambda ^ T g(x) $$

Where, 
$$f(x) = \frac{1}{2} \bar{x}^T G \bar{x} + g^T \bar{x} \quad \text{and} \quad g(\bar{x}) = G(\bar{x}) \Delta x$$

Please note that although the variables $G$ and $g$ are repeated, they have different meanings.

The hessian of the Lagrangian will be given by, 
$$ \nabla ^2 \mathcal{L} (x, \lambda) = \begin{bmatrix} \frac{\partial ^2 \mathcal{L}}{\partial x^2} & \frac{\partial ^2 \mathcal{L}}{\partial x \partial \lambda} \\ \frac{\partial ^2 \mathcal{L}}{\partial \lambda \partial x} & \frac{\partial ^2 \mathcal{L}}{\partial \lambda ^2}\end{bmatrix}$$

Here, we must ignore the second order derivatives of the constraints. This results in the Hessian of the Lagrangian becoming, 
$$ \nabla ^2 \mathcal{L} (x, \lambda) \approx  \nabla_{xx} ^2 \mathcal{L} (x)$$

Applying the same shortcut method to find the gradient and hessian, we can find that, 
$$ \nabla_{xx} ^2 \mathcal{L} (x) = 2 \cdot \begin{bmatrix} Q & 0 & 0 & \cdots \\ 0 & Q & 0 & \cdots \\ 0 & 0 & Q & \cdots \\ \vdots & \vdots & \vdots & \ddots\end{bmatrix}  $$

We shall use this logic to write the function for this computation, as shown below.

In [7]:
def compute_hessian_L():
    """Function to compute the hessian of the lagrangian, ignoring second order derivatives of the constraints

    Returns:
        np.ndarray: Hessian of the lagrangian
    """
    return 2 * block_diag(*([np.array([[20, 0, 0], [0, 0.2, 0], [0, 0, 0.2]])] * N))

#### 6. Solving the KKT System
Now that we have obtained all the prerequsites for forming the KKT System of equations, we shall now have to solve the QP, 
$$
\begin{equation}
\begin{aligned}
    & \min_{p} \frac{1}{2} p^T \nabla_{xx}^2 \mathcal{L}(x_k) p + p^T \nabla f(x_k) \\
    & \text{subject to } \nabla g(x_k)^T p + g(x_k) = 0
\end{aligned}
\end{equation}$$ 

Which is the same as solving

$$\begin{equation}
    \begin{bmatrix} 
        \nabla^2_{xx} \mathcal{L}(x_k) & \nabla g(x_k) \\
        \nabla g(x_k)^T & 0 
    \end{bmatrix}
    \begin{pmatrix} 
        p_k \\ 
        \lambda_{k+1} 
    \end{pmatrix} 
    = 
    \begin{pmatrix} 
        -\nabla f(x_k) \\ 
        -g(x_k) 
    \end{pmatrix}
\end{equation}$$

The below function was written based on this logic.

In [ ]:
def solve_KKT(bar_x: np.ndarray):
    lag_hess_mat = compute_hessian_L()
    const_jac_mat, const_mat = compute_G_g(bar_x=bar_x)
    cost_grad_mat = cost_grad(bar_x)
    LHS = np.block([[lag_hess_mat, const_jac_mat], [const_jac_mat.T, np.zeros(
        (const_jac_mat.T.shape[0], const_jac_mat.T.shape[0]))]])
    RHS = np.block([-cost_grad_mat, -const_mat])
    res = np.linalg.solve(LHS, RHS)
    Delta_x = res[:lag_hess_mat.shape[0]]
    Delta_lambda = res[lag_hess_mat.shape[0]:]

## Question 2: write a SQP solver with inequality constraints
Modify your SQP solver in order to enforce the additional constraint $-4 \leq u_n \leq 4$. 

In this case you will need to use a QP solver instead of the ``solve`` function. Please use the [qpsolvers](https://pypi.org/project/qpsolvers/) library (use ``pip install qpsolvers`` to get the latest version 4.4.0 and use ``cvxopt`` as QP solver). You may access the Lagrange multipliers of the solution following [this example](https://qpsolvers.github.io/qpsolvers/quadratic-programming.html#dual-multipliers).

Update the convergence checks accordingly (using the KKT condition for the nonlinear problem $\nabla_x L$). Also update the computation of the constraint violation by computing the amount of inequality constraint violation in absolute value (note that it should be zero when the constraint is satisfied).

Once you have a solution, make sure to check that it satisfies the constraints! You can also use the function ``pendulum.animate_robot`` to display the pendulum motion. Please answer the following questions:
1. How many iterations did it take?
2. Plot the solution (angle, velocity and control)
3. Plot the amont of constraint violation per iteration of the solver
4. Plot the cost per iteration of the solver
5. Plot $\alpha$ for each iteration of the solver
6. Compare this solution with the solution from Question 1. Are there any qualitative differences in the pendulum behavior? Did the solver converge faster or slower?

In [5]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython

from qpsolvers import solve_qp, Problem, solve_problem

import pendulum

In [6]:
# dt is defined here
print(f'we use the following dt={pendulum.dt}')

# and g here
print(f'we use the following g={pendulum.g}')

# you can use this animate function to display what the pendulum would do for a given sequence of control
N = 300
controls = np.zeros((N, 1))
x_init = np.array([[1.0], [0.]])
pendulum.animate_robot(x_init, controls.T)

we use the following dt=0.01
we use the following g=9.81


Your browser does not support the video tag.